In [1]:
import os
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import cv2


In [2]:
# 하이퍼파라미터
batch_size = 64
img_size = 48  # FER-2013이 보통 48x48
epochs = 30

train_dir = './data/train'
test_dir = './data/test'

In [3]:
# 기존에 쓰던 증강 옵션 + 추가 가능한 옵션
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],  # 밝기 범위 조절
    # channel_shift_range=20.0,   # 채널 값 변환 (흑백일 경우 무의미할 수도)
)

# 테스트셋은 과도한 증강 대신 rescale만
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 23308 images belonging to 5 classes.
Found 5834 images belonging to 5 classes.


In [4]:
num_classes = 5  # angry, fear, happy, sad, surprise

model = Sequential([
    Input(shape=(img_size, img_size, 1)),

    Conv2D(32, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),  # 24 x 24

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),  # 12 x 12

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),  # 6 x 6

    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),  # 3 x 3

    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 6, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 6, 6, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,014,917 (3.87 MB)

 Trainable params: 1,013,189 (3.87 MB)

 Non-trainable params: 1,728 (6.75 KB)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,      # 학습률을 절반으로 줄임
    patience=3,      # 3 epoch 동안 개선이 없으면 감소
    min_lr=1e-7,     # 학습률 하한선
    verbose=2
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=6,      # 6 epoch 동안 개선이 없으면 종료
    restore_best_weights=True,

)

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping],
    verbose=2
)


Epoch 1/50
365/365 - 28s - 77ms/step - accuracy: 0.2767 - loss: 1.8374 - val_accuracy: 0.3781 - val_loss: 1.4844 - learning_rate: 1.0000e-04
Epoch 2/50
365/365 - 28s - 77ms/step - accuracy: 0.2979 - loss: 1.7394 - val_accuracy: 0.4042 - val_loss: 1.4166 - learning_rate: 1.0000e-04
Epoch 3/50
365/365 - 29s - 81ms/step - accuracy: 0.3171 - loss: 1.6730 - val_accuracy: 0.4242 - val_loss: 1.3780 - learning_rate: 1.0000e-04
Epoch 4/50
365/365 - 29s - 79ms/step - accuracy: 0.3377 - loss: 1.6249 - val_accuracy: 0.4450 - val_loss: 1.3420 - learning_rate: 1.0000e-04
Epoch 5/50
365/365 - 28s - 77ms/step - accuracy: 0.3580 - loss: 1.5731 - val_accuracy: 0.4374 - val_loss: 1.3438 - learning_rate: 1.0000e-04
Epoch 6/50
365/365 - 27s - 74ms/step - accuracy: 0.3771 - loss: 1.5081 - val_accuracy: 0.4865 - val_loss: 1.2462 - learning_rate: 1.0000e-04
Epoch 7/50
365/365 - 27s - 74ms/step - accuracy: 0.3930 - loss: 1.4721 - val_accuracy: 0.4791 - val_loss: 1.2659 - learning_rate: 1.0000e-04
Epoch 8/50
36

In [3]:
!pip install deepface


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from deepface.extendedmodels import Emotion

emotion_model = Emotion.loadModel()

ModuleNotFoundError: No module named 'deepface.extendedmodels'